In [ ]:
# LiPF6結晶とH2Oの混合系を350Kと800KでMD実行し、HF等の生成を追跡することでMatlantisポテンシャルの妥当性を検証する。
import numpy as np
import os
from time import perf_counter
from typing import Dict, Any

from ase import Atoms
from ase.build import molecule
from ase.io import Trajectory, read
from ase import units

# --- Matlantis Features のコアコンポーネントをインポート ---
try:
    from matlantis_features.features.md import (
        MDFeature,
        ASEMDSystem,
        MDExtensionBase,
        NVTBerendsenIntegrator,
    )
    from matlantis_features.utils.calculators import pfp_estimator_fn
except ImportError:
    print("エラー: 'matlantis_features' が見つかりません。")
    exit()

# --- ★ ご提示いただいた Utils から PrintWriteLog を引用 ★ ---
try:
    # (ご自身のパッケージ名に合わせて 'lib2_utils' の部分を変更してください)
    from lib2_utils.md_utils import PrintWriteLog
except ImportError:
    print("="*50)
    print("警告: 'from lib2_utils.md_utils import PrintWriteLog' に失敗しました。")
    print("続行のため、PrintWriteLog をこの場で再定義します。")
    print("="*50)
    
    # --- lib2_utils が見つからない場合のフォールバック (内容はご提示のコードと同じ) ---
    class PrintWriteLog(MDExtensionBase):
        def __init__(self, fname: str, dirout: str = '.', stdout: bool = False):
            self.fname = fname
            self.dirout = dirout
            self.t_start = perf_counter()
            self.stdout = stdout
        def __call__(self, system, integrator):
            n_step = system.current_total_step
            sim_time_ps = system.current_total_time / 1000.0
            E_tot = system.ase_atoms.get_total_energy()
            E_pot = system.ase_atoms.get_potential_energy()
            E_kin = system.ase_atoms.get_kinetic_energy()
            temp = system.ase_atoms.get_temperature()
            try:
                density = system.ase_atoms.get_masses().sum() / units.mol / (
                    system.ase_atoms.cell.volume * (1e-8**3)
                )
            except: density = 0.0
            calc_time = (perf_counter() - self.t_start) / 60.
            if n_step == 0:
                hdr = ('step,time[ps],E_tot[eV],E_pot[eV],E_kin[eV],'
                       'T[K],density[g/cm3],calc_time[min]')
                with open(f'{self.dirout}/{self.fname}.log', 'w') as f_log:
                    f_log.write(f'{hdr}\n')
            line = (f'{n_step:8d},{sim_time_ps:7.2f},'
                    f'{E_tot:11.4f},{E_pot:11.4f},{E_kin:9.4f},'
                    f'{temp:8.2f},{density:7.3f},{calc_time:8.2f}')
            with open(f'{self.dirout}/{self.fname}.log', 'a') as f_log:
                f_log.write(f'{line}\n')
            if self.stdout: print(f"MD LOG: {line}")
  
# -------------------------------------------------------------------------
# 【修正版 v2】反応追跡用のカスタムロガー (TrackReaction)
# (距離行列を一括計算してスライシングする方式に変更)
# -------------------------------------------------------------------------

class TrackReaction(MDExtensionBase):
    """
    MDシミュレーション中に反応生成物（HF, LiF, PO）を追跡するクラス
    """
    def __init__(self, fname: str, dirout: str = '.', stdout: bool = True):
        self.fname = fname
        self.dirout = dirout
        self.stdout = stdout
        self.log_path = f'{self.dirout}/{self.fname}_reaction.log'

    def __call__(self, system, integrator):
        n_step = system.current_total_step
        sim_time_ps = system.current_total_time / 1000.0
        atoms = system.ase_atoms

        if n_step == 0:
            hdr = 'step,time[ps],n_HF,n_LiF,n_PO'
            with open(self.log_path, 'w') as f:
                f.write(f'{hdr}\n')

        # --- 1. 原子インデックスの取得 ---
        h_idx = [a.index for a in atoms if a.symbol == 'H']
        f_idx = [a.index for a in atoms if a.symbol == 'F']
        li_idx = [a.index for a in atoms if a.symbol == 'Li']
        p_idx = [a.index for a in atoms if a.symbol == 'P']
        o_idx = [a.index for a in atoms if a.symbol == 'O']

        n_hf = n_lif = n_po = 0

        # --- 2. 全原子間距離行列の一括計算 (mic=Trueで周期境界考慮) ---
        # これで形状不一致のエラーは絶対に起きません
        try:
            # 全原子 x 全原子 の距離行列を取得
            all_dists = atoms.get_all_distances(mic=True)
            
            # --- 3. 必要なペアを抜き出してカウント (numpyスライシング) ---
            
            # HF生成（< 1.0Å）
            if h_idx and f_idx:
                # h_idx行、f_idx列 のサブ行列を抜き出す
                hf_dists = all_dists[np.ix_(h_idx, f_idx)]
                n_hf = (hf_dists < 1.0).sum()

            # LiF生成（< 1.8Å）
            if li_idx and f_idx:
                lif_dists = all_dists[np.ix_(li_idx, f_idx)]
                n_lif = (lif_dists < 1.8).sum()

            # POF3のP=O結合（< 1.6Å）
            if p_idx and o_idx:
                po_dists = all_dists[np.ix_(p_idx, o_idx)]
                n_po = (po_dists < 1.6).sum()

        except Exception as e:
            print(f"REACTION LOG ERROR: {e}")
            # エラー時もログが止まらないように0で進める
            pass 

        # --- 4. 書き込み ---
        line = f'{n_step:8d},{sim_time_ps:7.2f},{n_hf:5d},{n_lif:5d},{n_po:5d}'
        with open(self.log_path, 'a') as f:
            f.write(f'{line}\n')
        if self.stdout:
            print(f"REACTION LOG: {line}")
# -------------------------------------------------------------------------
# 実行部 (run_constant_temp_md のロジックを展開)
# -------------------------------------------------------------------------

if __name__ == "__main__":
    
    # === 1. 検証用システムの構築 (CIFファイルを使用) ===
    
    # ★★★ ご指定のCIFファイルのパス ★★★
    cif_path = "/home/jovyan/Kaori/MD/input/LiPF6.cif"
    
    try:
        lipf6_crystal = read(cif_path)
        print(f"--- CIFファイル読み込み成功: {cif_path}")
        print(f"--- 読み込んだ構造: {lipf6_crystal.get_chemical_formula()}")
    except FileNotFoundError:
        print(f"エラー: CIFファイルが見つかりません: {cif_path}")
        print("スクリプトを終了します。")
        exit()
    except Exception as e:
        print(f"エラー: CIFファイルの読み込みに失敗しました: {e}")
        exit()

    
    h2o_template = molecule('H2O')
    system = lipf6_crystal.copy()
    
    cell_size = 25.0 # 25Åの真空ボックス (結晶が約12Åのため余裕を持つ)
    n_water = 20     # 20分子の水を追加
    
    # 結晶を大きな真空ボックスの中央に配置
    system.set_cell([cell_size, cell_size, cell_size], scale_atoms=False)
    system.center()
    
    # 真空ボックス内にH₂O分子をランダムに配置
    # (より高度な充填には lib2_utils.structure_utils が使えるかもしれません)
    print(f"--- {n_water} 分子の H2O をランダムに配置します...")
    for i in range(n_water):
        h2o = h2o_template.copy()
        # セル内にランダムな座標を生成
        pos = np.random.rand(3) * cell_size
        h2o.translate(pos)
        system.extend(h2o)
        
    system.set_pbc(True)
    print(f"--- システム構築完了: {system.get_chemical_formula()}")

    # === 2. MDパラメータの定義 ===
    
    # ★★★ ご自身の環境に合わせてPFPモデルバージョンを指定してください ★★★
    MODEL_VERSION = 'v7.0.0' # (md_utils.py に記載されていたデフォルト値)
    CALC_MODE = 'CRYSTAL_U0' # (md_utils.py に記載されていたデフォルト値)
    
    TIMESTEP_FS = 0.5
    SIM_TIME_PS = 20.0
    N_STEPS = int(SIM_TIME_PS * 1000 / TIMESTEP_FS) # 40000 ステップ

    base_params = {
        'timestep': TIMESTEP_FS,
        'n_run': N_STEPS,
        'model_version': MODEL_VERSION,
        'calc_mode': CALC_MODE,
        'dirout': 'step3_validation_md_cif', # 出力先ディレクトリを変更
        'logger_interval': 100,
        'traj_freq': 100,
    }
    os.makedirs(base_params['dirout'], exist_ok=True)

    # --- 350K (低温) パラメータ ---
    params_350K = base_params.copy()
    params_350K.update({
        'temperature': 350.0,
        'fname': 'md_350K_LiPF6_crystal_H2O',
        'atoms': system.copy()
    })
    
    # --- 800K (高温・陽性対照) パラメータ ---
    params_800K = base_params.copy()
    params_800K.update({
        'temperature': 800.0,
        'fname': 'md_800K_LiPF6_crystal_H2O',
        'atoms': system.copy()
    })

    # === 3. シミュレーションの実行 ===
    # (run_constant_temp_md の中身を展開し、extensionsをカスタマイズ)

    print("\n--- ポテンシャル検証MD（ステップ3、CIF使用版）を開始します ---")
    
    # 2つのシミュレーション（350Kと800K）を順番に実行
    for md_params in [params_350K, params_800K]:
        
        # --- ここからが run_constant_temp_md のロジック ---
        estimator_fn = pfp_estimator_fn(
            model_version=md_params['model_version'],
            calc_mode=md_params['calc_mode'],
        )
        system_md = ASEMDSystem(
            atoms=md_params['atoms'],
            step=0,
            time=0.0,
        )
        integrator = NVTBerendsenIntegrator(
            timestep=md_params['timestep'],
            temperature=md_params['temperature'],
            taut=100.,
            fixcm=True,
        )
        md = MDFeature(
            integrator=integrator,
            n_run=md_params['n_run'],
            show_progress_bar=True,
            show_logger=False, # カスタムロガーを使うためFalse
            logger_interval=md_params['logger_interval'],
            estimator_fn=estimator_fn,
            traj_file_name=f"{md_params['dirout']}/{md_params['fname']}.traj",
            traj_freq=md_params['traj_freq'],
        )

        print(f"\n--- MD計算を開始: {md_params['fname']} ---")
        print(f"ステップ数: {md_params['n_run']} ({md_params['n_run'] * md_params['timestep']/1000:.1f} ps), 温度: {md_params['temperature']:.2f} K")

        # --- 2つの拡張機能（ロガー）を準備 ---
        
        # 1. utilsからインポートした標準ロガー
        std_logger = (
            PrintWriteLog(
                fname=md_params['fname'], 
                dirout=md_params['dirout'], 
                stdout=True # utilsのデフォルトはFalseですが、確認のためTrueにします
            ),
            md_params['logger_interval']
        )
        
        # 2. このスクリプトで定義した反応追跡ロガー
        reaction_tracker = (
            TrackReaction(
                fname=md_params['fname'], 
                dirout=md_params['dirout'], 
                stdout=True
            ),
            md_params['logger_interval']
        )

        # 2つのロガーを extensions リストとして渡してMD実行
        md_results = md(
            system=system_md,
            extensions=[std_logger, reaction_tracker]
        )
        print(f"--- MD計算が完了: {md_params['fname']} ---")
        # --- run_constant_temp_md のロジックここまで ---

    # === 4. 完了報告 ===
    print("\n" + "="*50)
    print("           ポテンシャル検証MD（CIF版）完了")
    print("="*50)
    print(f"出力ディレクトリ: {base_params['dirout']}")
    print("\n次にやるべきこと:")
    print("1. 'md_350K_LiPF6_crystal_H2O_reaction.log' と")
    print("   'md_800K_LiPF6_crystal_H2O_reaction.log' を開きます。")
    print("2. n_HF, n_LiF, n_PO の列を比較してください。")
    print("\n【期待される結果】:")
    print(" - 350K: n_HF は 0 のまま。")
    print(" - 800K: n_HF が時間経過とともに増加する。")
    print("   (結晶の溶解 -> PF6-の加水分解 という、より複雑なプロセスを観測)")
    print("="*50)

警告: 'from lib2_utils.md_utils import PrintWriteLog' に失敗しました。
続行のため、PrintWriteLog をこの場で再定義します。
--- CIFファイル読み込み成功: /home/jovyan/Kaori/MD/input/LiPF6.cif
--- 読み込んだ構造: F18Li3P3
--- 20 分子の H2O をランダムに配置します...
--- システム構築完了: H40F18Li3O20P3

--- ポテンシャル検証MD（ステップ3、CIF使用版）を開始します ---

--- MD計算を開始: md_350K_LiPF6_crystal_H2O ---
ステップ数: 40000 (20.0 ps), 温度: 350.00 K


  0%|          | 0/40000 [00:00<?, ?it/s]

The MD trajectory will be saved at /home/jovyan/Kaori/MD/LiB_2/structure/step3_validation_md_cif/md_350K_LiPF6_crystal_H2O.traj.
Note: The max disk size of /home/jovyan is about 98G.
/home/jovyan/.py313/lib/python3.13/site-packages/ase/md/nvtberendsen.py:98: RuntimeWarning: divide by zero encountered in scalar divide
  1.0 + (self.temperature / old_temperature - 1.0) * tautscl


MD LOG:        0,   0.00,  -237.6954,  -237.6954,   0.0000,    0.00,  0.087,    0.00
REACTION LOG:        0,   0.00,    1,    0,    0
MD LOG:      100,   0.05,  -250.0579,  -274.2433,  24.1854, 2227.46,  0.087,    0.49
REACTION LOG:      100,   0.05,    0,    5,    0
MD LOG:      200,   0.10,  -258.9632,  -275.9050,  16.9418, 1560.33,  0.087,    0.99
REACTION LOG:      200,   0.10,    0,    4,    0
MD LOG:      300,   0.15,  -264.7916,  -278.2426,  13.4510, 1238.82,  0.087,    1.47
REACTION LOG:      300,   0.15,    0,    2,    0
MD LOG:      400,   0.20,  -269.2504,  -280.9987,  11.7484, 1082.02,  0.087,    1.97
REACTION LOG:      400,   0.20,    0,    2,    0
MD LOG:      500,   0.25,  -272.5022,  -281.3217,   8.8195,  812.27,  0.087,    2.38
REACTION LOG:      500,   0.25,    0,    2,    0
MD LOG:      600,   0.30,  -275.4663,  -284.8341,   9.3678,  862.77,  0.087,    2.86
REACTION LOG:      600,   0.30,    1,    2,    0
MD LOG:      700,   0.35,  -277.6447,  -283.5876,   5.9429,  5